In [73]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
class color:
    PURPLE = '\033[95m'
    CYAN = '\033[96m'
    DARKCYAN = '\033[36m'
    BLUE = '\033[94m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    RED = '\033[91m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'
    END = '\033[0m'
pd.set_option("display.max_columns", 180)
from tqdm.notebook import tqdm

In [8]:
def normalize(data):
    df=data.copy()
    for column in df.columns[11:]:
        minim = df[column].min()
        maxim = df[column].max()
        df[column] = (df[column] - minim) / (maxim - minim)
    return df

In [64]:
def compute_similarity(data, player, min_minutes=100):
    df = data[data.minutesPlayed>=min_minutes].reset_index(drop=True)
    df = normalize(df)
    j = df[df['playerId']==player].index.values[0]    
    similarity = df.copy()
    for row in range(0,len(df)):
        for column in df.columns[11:]:
            similarity.at[row,column] = abs(df.loc[row,column] - df.loc[j,column])
            
    similarity['Distance']= similarity.iloc[:,11:].mean(axis=1) 
    similarity['SimilarityIndex'] = round((1 - similarity['Distance'])*100, 3)
    similarity = similarity[['playerId', 
                             'shortName', 
                             'team', 
                             'position', 
                             'age',
                             'height', 
                             'foot', 
                             'minutesPlayed',
                             'marketValue', 
                             'goals', 
                             'minutesPlayed', 
                             'SimilarityIndex']]
    similarity = similarity.sort_values(by='SimilarityIndex', ascending=False).iloc[1:,:].reset_index(drop=True)
    return similarity

In [65]:
df = pd.read_csv('final_data/full_data.csv')

In [70]:
df.head(3)

,playerId,shortName,team,position,height,foot,age,value,marketValue,minutesPlayed,goals,X_pass_mean,X_pass_std,Y_pass_mean,Y_pass_std,forward_passes,smart_passes,pass_end_z1,pass_end_z2,pass_end_z3,pass_end_z4,pass_end_z5,pass_end_z6,pass_end_z7,pass_end_z8,pass_end_z9,pass_reception_z1,pass_reception_z2,pass_reception_z3,pass_reception_z4,pass_reception_z5,pass_reception_z6,pass_reception_z7,pass_reception_z8,pass_reception_z9,dribble_z1,dribble_z2,dribble_z3,dribble_z4,dribble_z5,dribble_z6,dribble_z7,dribble_z8,dribble_z9,cross_z1,cross_z2,cross_z3,cross_z4,shot_z1,shot_z2,shot_z3,shot_z4,shot_wleft,shot_wright,shot_whead,defense_z1,defense_z2,defense_z3,defense_z4,defense_z5,defense_z6,defense_z7,defense_z8,defense_z9,defense_z10,defense_z11,defense_z12,clearances_per90,interceptions_per90,tackles_per90,loose_ball_won_per90,final_third_recovery_per90,air_success,headers_per90,ABCD,BACD,BCAD,BCDA,ABAB,BABA,ABAC,BABC,BCBA,ABCA,BACB,BCAB,ABCB,BACA,BCAC,flow_centrality
0,36,T. Alderweireld,Tottenham Hotspur,Defender,187,right,29,40000000,"$40,000,000",1170,0,0.924887,0.730094,0.847238,0.885403,0.735009,0.016004,0.203639,0.499631,0.449772,0.207642,0.372417,0.489201,0.117808,0.047712,0.189873,0.043631,0.292019,0.573938,0.048657,0.142138,0.362644,0.003838,0.021116,0.040318,0.000000,0.000000,0.076720,0.000000,0.0,0.077279,0.000000,0.000000,0.000000,0.011867,0.097201,0.000000,0.000000,0.157396,0.080316,0.044916,0.000000,0.100000,0.700000,0.200000,0.169412,0.274091,0.381429,0.141429,0.146667,0.825000,0.055556,0.000000,0.317059,0.074545,0.000000,0.111818,0.407240,0.411831,0.304191,0.173022,0.085897,0.494215,0.240531,0.585529,0.630912,0.646216,0.495600,0.151852,0.142361,0.358923,0.265210,0.468278,0.304363,0.406039,0.297506,0.451790,0.368322,0.229543,0.684922
1,48,J. Vertonghen,Tottenham Hotspur,Defender,189,left,31,32000000,"$32,000,000",3204,0,0.785901,0.612459,0.714157,0.734989,0.716262,0.017950,0.389578,0.564472,0.182470,0.547623,0.389957,0.138710,0.136331,0.056063,0.042593,0.616479,0.232474,0.005113,0.403902,0.207571,0.000000,0.036828,0.010279,0.000000,0.158638,0.080936,0.000000,0.024737,0.0,0.000000,0.034296,0.000000,0.016952,0.056336,0.000000,0.031471,0.000000,0.134111,0.102651,0.008201,0.017207,0.652174,0.173913,0.173913,0.339869,0.264362,0.088294,0.752976,0.267361,0.078125,0.424383,0.230903,0.020016,0.064710,0.025735,0.025884,0.421348,0.616761,0.463926,0.307324,0.144288,0.760137,0.356334,0.578393,0.572947,0.583926,0.429053,0.166355,0.098773,0.300782,0.292644,0.427686,0.290209,0.249532,0.241044,0.431304,0.286696,0.246211,0.696820
2,54,C. Eriksen,Tottenham Hotspur,Midfielder,180,right,26,80000000,"$80,000,000",3226,10,0.579671,0.482398,0.485983,0.632286,0.399005,0.294509,0.134419,0.190884,0.226423,0.359003,0.702387,0.407117,0.324459,0.268783,0.343178,0.078603,0.091686,0.128107,0.280846,0.783408,0.356000,0.097460,0.071774,0.169104,0.047656,0.000000,0.051939,0.089173,0.0,0.052318,0.092723,0.133684,0.000000,0.111903,0.287056,0.062513,0.151882,0.475702,0.145644,0.097740,0.136719,0.337500,0.650000,0.012500,0.076688,0.045118,0.060544,0.279365,0.244444,0.268519,0.339506,0.440741,0.512418,0.179461,0.137255,0.331313,0.068925,0.279979,0.188199,0.230089,0.461066,0.340909,0.136412,0.373804,0.418447,0.471218,0.494120,0.165220,0.180709,0.246996,0.326195,0.298297,0.401682,0.290931,0.364159,0.313664,0.288256,0.334809,0.665739


## Simple program that allows to search Top N most similar players to a reference player

In [69]:
i=0
while i<1:
    j=0
    while j<1:
        player_string = input('\n Search for player: ')
        search = df[(df['shortName'].str.contains(player_string, case=False))][['playerId', \
                                                                                                'shortName', 'team', 'position', 'marketValue']]
        if search.shape[0]==0:
            print('\n No results found. Please try again. Do not include the first name of the player, but only his last name or nickname.')
            j=j
        else:
            j=j+1
    display.display(search)
    playerId = input('\n Enter the playerId of the player you were looking for, or enter 0 for a new search: ')
    try:
        playerId = int(playerId)
    except:
        playerId = input('\n Please enter an integer number (playerId): ')
        playerId = int(playerId)
    if playerId==0:
        i=i
    else:
            i=i+1
value = input("\n Look for similar players for less than (in millions of euros, type 'Any' if no money filter is desired): ")
if (value!='Any') and (value!='any'):
    try:
        value = float(value)
        value = value*1000000
    except:
        value = input('\n Please enter a number (market value in millions of Euros): ')
        value = float(value)
        value = value*1000000
        
if df[df['playerId']==playerId]['playerId'].any():
    print('\nComputing Similarity. Please wait...')
    df=normalize(df)
    similarity = compute_similarity(df,playerId,900)
    print(color.BOLD + "\nThe 5 players most similar to {} for less than {} Euros are:".format(df[df['playerId']==playerId]['shortName'].iloc[0], value) + color.END)
    if (value!='Any') and (value!='any'):
        similarity = similarity[similarity['value']<=value]
        aux = similarity.reset_index()
        num = aux.iloc[0,aux.columns.get_loc('index')]
        print('(Note: There are {} players more similar to {} than {} for a higher price)'.format(num, all_players[all_players['playerId']==playerId]['shortName'].iloc[0], \
                                                                                                 aux.iloc[0,2]))
    show = similarity[['playerId', 'shortName', 'team', 'position', 'age','height', 'foot', 'marketValue', 'goals', 'minutesPlayed', 'SimilarityIndex']].reset_index(drop=True)
    display.display(show.head(5))
else:
    print('\nPlayer does not exist or did not play the minimum 900 minutes across the 2017-18 season. \nPlease try a different player.')


 Search for player: 3359

 No results found. Please try again. Do not include the first name of the player, but only his last name or nickname.

 Search for player: messi


,playerId,shortName,team,position,marketValue
91,3359,L. Messi,Barcelona,Forward,"$180,000,000"



 Enter the playerId of the player you were looking for, or enter 0 for a new search: 3359

 Look for similar players for less than (in millions of euros, type 'Any' if no money filter is desired): Any

Computing Similarity. Please wait...

The 5 players most similar to L. Messi for less than Any Euros are:


,playerId,shortName,team,position,age,height,foot,marketValue,goals,minutesPlayed,minutesPlayed,SimilarityIndex
0,89186,P. Dybala,Juventus,Forward,24,177,left,"$110,000,000",16,2359,2359,88.298
1,25707,E. Hazard,Chelsea,Forward,27,173,right,"$110,000,000",10,2432,2432,86.228
2,20820,J. Ilicic,Atalanta,Forward,30,190,left,"$13,000,000",9,2002,2002,85.554
3,3840,Iago Aspas,Celta de Vigo,Forward,31,176,left,"$25,000,000",19,2927,2927,85.279
4,28115,N. Fekir,Olympique Lyonnais,Midfielder,25,173,left,"$60,000,000",12,2482,2482,85.199
